# Week 3 - Toronto Neighbourhoods

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Import additional web scraping libraries

In [16]:
import urllib.request
import time
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## Load and explore the data

In [17]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

response

<Response [200]>

### Source the response html code:

In [82]:
soup = BeautifulSoup(response.text, 'html.parser')

# print(soup.prettify())

### Find postcode table in html code:

In [88]:
table_tag = soup.findAll('table')[0]

postal_code_table = soup.find('table',{'class':'wikitable sortable'})

print(postal_code_table)

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

### Extract rows from table

In [98]:
rows = postal_code_table.find_all('tr')

postal_codes = []
boroughs = []
neighbourhoods = []

for row in rows:
#     print(row)
#     print(" ")
    cells = row.find_all('td')

    if len(cells) > 1:
            postal_code = cells[0]
            borough = cells[1]
            neighbourhood = cells[2]

            # Skip 'Not assigned' Boroughs
            if borough.text.strip()  != 'Not assigned':  
                postal_codes.append(postal_code.text.strip())
                boroughs.append(borough.text.strip())
                
                if neighbourhood.text.strip()  == 'Not assigned':  
                    neighbourhoods.append(borough.text.strip())
                else:
                    neighbourhoods.append(neighbourhood.text.strip())

### Create Pandas Dataframe from lists

In [100]:
postal_code_df = pd.DataFrame(zip(postal_codes, boroughs, neighbourhoods), columns =['Postal Code', 'Borough', 'Neighbourhood'])

# postal_code_df = postal_code_df.sort_values(by = ['Postal Code'], ascending = True)

print(postal_code_df)

    Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
5           M9A         Etobicoke   
6           M1B       Scarborough   
7           M3B        North York   
8           M4B         East York   
9           M5B  Downtown Toronto   
10          M6B        North York   
11          M9B         Etobicoke   
12          M1C       Scarborough   
13          M3C        North York   
14          M4C         East York   
15          M5C  Downtown Toronto   
16          M6C              York   
17          M9C         Etobicoke   
18          M1E       Scarborough   
19          M4E      East Toronto   
20          M5E  Downtown Toronto   
21          M6E              York   
22          M1G       Scarborough   
23          M4G         East York   
24          M5G  Downtown Toronto   
25          M6G  Downtown Toronto   
2

### Check dataframe shape

In [104]:
postal_code_df.shape

(103, 3)